In [ ]:
# Retrieve cached data

# Retrieve cached data
wh_acctcommon_me = pd.read_csv(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\caching\wh_acctcommon_me.csv')
wh_loans_me = pd.read_csv(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\caching\wh_loans_me.csv')
wh_acctloan_me = pd.read_csv(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\caching\wh_acctloan_me.csv')
wh_acct_me = pd.read_csv(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\caching\wh_acct_me.csv')
wh_prop = pd.read_csv(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\caching\wh_prop.csv')
wh_prop2 = pd.read_csv(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\caching\wh_prop2.csv')

# Transforming the data
main_loan_data = src.transformations.joining.join_loan_tables(wh_acctcommon_me, wh_acctloan_me, wh_loans_me, wh_acct_me)
property_data = src.transformations.joining.join_prop_tables(wh_prop, wh_prop2)

# # Calculated fields & data cleaning
main_loan_data = src.transformations.calculations.append_total_exposure_field(main_loan_data)
main_loan_data = src.transformations.calculations.cleaning_loan_data(main_loan_data)

# Consolidate loan data & property data
single_prop_data = src.transformations.joining.consolidation_with_one_prop(main_loan_data, property_data)
multiple_prop_data = src.transformations.joining.consolidation_with_multiple_props(main_loan_data, property_data)

# Sort data
single_prop_data = single_prop_data.sort_values(by=['Total Exposure','acctnbr'], ascending=[False, True])
multiple_prop_data = multiple_prop_data.sort_values(by=['Total Exposure','acctnbr'], ascending=[False, True])

# Writing loan data with single property data to excel
single_prop_data_file_path = Path(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\single_property_per_loan.xlsx')
single_prop_data.to_excel(single_prop_data_file_path, engine='openpyxl', index=False)


# Writing loan data with multiple property data to excel
multiple_prop_data_file_path= Path(r'Z:\Chad Projects\Monthly Reports\Automated Linda Reports\NonOwnerOcc\Production\output\multiple_property_per_loan.xlsx')
multiple_prop_data.to_excel(multiple_prop_data_file_path, engine='openpyxl', index=False)

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
wh_acctcommon = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\wh_acctcommon.csv'))
wh_acctloan = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\wh_acctloan.csv'))
wh_loans = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\wh_loans.csv'))
acctpropins = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\acctpropins.csv'))
wh_prop = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\wh_prop.csv'))
wh_prop2 = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\wh_prop2.csv'))
acctloanlimithist = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\acctloanlimithist.csv'))
wh_inspolicy = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\wh_inspolicy.csv'))
persaddruse = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\persaddruse.csv'))
orgaddruse = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\orgaddruse.csv'))


In [3]:
def join_loan_tables(wh_acctcommon, wh_acctloan, wh_loans):
    """
    Merging core loan tables on acctnbr

    Args:
        wh_acctcommon (pd.DataFrame): (month end) from COCCDM
        wh_acctloan (pd.DataFrame): (month end) from COCCDM
        wh_loans (pd.DataFrame): (month end) from COCCDM
    
    
    Returns:
        df (pd.DataFrame): Combined loan tables into a df
    """
    # Pre-merge validation
    assert wh_acctcommon['acctnbr'].is_unique, "Duplicates exist"
    assert wh_acctloan['acctnbr'].is_unique, "Duplicates exist"
    assert wh_loans['acctnbr'].is_unique, "Duplicates exist"

    # Merging
    df = pd.merge(wh_acctcommon, wh_acctloan, on='acctnbr', how='left')
    df = pd.merge(df, wh_loans, on='acctnbr', how='left')

    return df

def join_prop_tables(wh_prop, wh_prop2):
    """
    Joining property tables together in a left-join on wh_prop.

    Args:
        wh_prop (pd.DataFrame): OSIBANK.WH_PROP
        wh_prop2 (pd.DataFrame): OSIBANK.WH_PROP2
    
    Returns:
        df (pd.DataFrame): Joined property tables as a df
    """
    wh_prop2 = wh_prop2.drop_duplicates(subset='propnbr', keep='first')

    # Tests
    assert wh_prop2['propnbr'].is_unique, "Duplicates exist"

    wh_prop2 = wh_prop2.drop(columns=['acctnbr']).copy()

    # Merge
    df = pd.merge(wh_prop, wh_prop2, on='propnbr', how='left')

    return df

In [4]:
main_data = join_loan_tables(wh_acctcommon, wh_acctloan, wh_loans)

In [5]:
prop_data = join_prop_tables(wh_prop, wh_prop2)

In [6]:
orgaddruse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33176 entries, 0 to 33175
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   orgnbr     33176 non-null  int64 
 1   addrnbr    33176 non-null  int64 
 2   addrusecd  33176 non-null  object
dtypes: int64(2), object(1)
memory usage: 777.7+ KB


In [8]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24203 entries, 0 to 24202
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   acctnbr              24203 non-null  int64  
 1   effdate              24203 non-null  object 
 2   ownersortname        24203 non-null  object 
 3   nameaddr1            24203 non-null  object 
 4   nameaddr2            24203 non-null  object 
 5   primaryownercity     24185 non-null  object 
 6   primaryownerstate    24185 non-null  object 
 7   primaryownerzipcd    24185 non-null  float64
 8   notebal              24203 non-null  float64
 9   bookbalance          24203 non-null  float64
 10  mjaccttypcd          24203 non-null  object 
 11  currmiaccttypcd      24203 non-null  object 
 12  curracctstatcd       24203 non-null  object 
 13  taxrptforpersnbr     21020 non-null  float64
 14  taxrptfororgnbr      3183 non-null   float64
 15  loanofficer          24203 non-null 

In [9]:
import numpy as np
import pandas as pd

def append_total_exposure_field(df):
    """ 
    Single Obligor Exposure Calculation
    
    Args:
        df: loan_data is loaded in
    
    Returns:
        df: loan_data is returned with new fields appended
        
    Operations:
        bookbalance -> if currmiaccttypcd == 'CM45', use notebal, else bookbalance
            - Tax Exempt bonds always have $0 as book balance so adjustment is made
        net balance == bookbalance - cobal
            - BCSB balance - Charged off amount (COBAL)
        net available == available balance amount * (1 - total pct sold)
        net collateral reserve == collateral reserve * (1 - total pct sold)
        total exposure == net balance + net available + net collateral reserve
    """
    # QA test
    list_of_numeric = ['bookbalance','notebal','availbalamt','totalpctsold','noteopenamt','creditlimitamt','noteintrate','cobal','credlimitclatresamt']
    for col in list_of_numeric:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    def convert_to_float(value):
        try:
            return float(value)
        except:
            return None
        
    for col in list_of_numeric:
        df[col] = df[col].apply(convert_to_float)
    
    # Tax Exempt bonds always have $0 Book Balance so need to take NOTEBAL
    df['bookbalance'] = np.where(df['currmiaccttypcd'].isin(['CM45']), df['notebal'], df['bookbalance'])
    df['Net Balance'] = df['bookbalance'] - df['cobal']
    df['Net Available'] = df['availbalamt'] * (1 - df['totalpctsold'])
    df['Net Collateral Reserve'] = df['credlimitclatresamt'] * (1 - df['totalpctsold'])
    df['Total Exposure'] = df['Net Balance'] + df['Net Available'] + df['Net Collateral Reserve']
    return df

def cleaning_loan_data(df):
    """
    Additional cleaning of the df dataset, excluding ACH manager products and converting datetime fields to appropriate data type
    
    Args:
        df (pd.DataFrame)

    Returns:
        df (pd.DataFrame)
    """
    # Exclude ACH Manager products
    df = df.loc[df['currmiaccttypcd'] != 'CI07'].copy()

    # Convert datetime fields
    date_fields = ['origdate']
    df[date_fields] = df[date_fields].apply(pd.to_datetime)

    # Create calculated field for total original loan amount
    df['orig_ttl_loan_amt'] = np.where(df['noteopenamt'] == 0, df['creditlimitamt'], df['noteopenamt'])

    return df

In [10]:
main_data = append_total_exposure_field(main_data)
main_data = cleaning_loan_data(main_data)

In [11]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23593 entries, 0 to 24202
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   acctnbr                 23593 non-null  int64         
 1   effdate                 23593 non-null  object        
 2   ownersortname           23593 non-null  object        
 3   nameaddr1               23593 non-null  object        
 4   nameaddr2               23593 non-null  object        
 5   primaryownercity        23575 non-null  object        
 6   primaryownerstate       23575 non-null  object        
 7   primaryownerzipcd       23575 non-null  float64       
 8   notebal                 23593 non-null  float64       
 9   bookbalance             23593 non-null  float64       
 10  mjaccttypcd             23593 non-null  object        
 11  currmiaccttypcd         23593 non-null  object        
 12  curracctstatcd          23593 non-null  object     

In [12]:
acctpropins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7960 entries, 0 to 7959
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   acctnbr               7960 non-null   int64 
 1   propnbr               7960 non-null   int64 
 2   intrpolicynbr         7960 non-null   int64 
 3   effdate               7960 non-null   object
 4   inactivedate          2 non-null      object
 5   datelastmaint         7960 non-null   object
 6   escrowyn              7960 non-null   object
 7   lenderfundedyn        7960 non-null   object
 8   escdisbchangeeffdate  6916 non-null   object
dtypes: int64(3), object(6)
memory usage: 559.8+ KB


In [13]:
wh_acctcommon[wh_acctcommon['acctnbr'] == 150439381]

,acctnbr,effdate,ownersortname,nameaddr1,nameaddr2,primaryownercity,primaryownerstate,primaryownerzipcd,notebal,bookbalance,mjaccttypcd,currmiaccttypcd,curracctstatcd,taxrptforpersnbr,taxrptfororgnbr,loanofficer,acctofficer,contractdate,noteopenamt,noteintrate
5492,150439381,2025-01-28,"KOLEV, NICKOLAY K.",NICKOLAY K. KOLEV,EVELYN R. ENCINAS KOLEV,SEEKONK,MA,2771.0,568657.09,568657.09,MTG,MG29,ACT,1112310.0,NaN,DAVID C. TIPPING,NaN,2020-03-12,618750.0,0.035


In [14]:
wh_inspolicy[wh_inspolicy['intrpolicynbr'] == 5698]

,intrpolicynbr,rundate,insorgnbr,instypcd,extpolicynbr,agentorgnbr,premcalperiodcd,premamt,effdate,expiredate,policydocnbr,inactivedate,firstmonthcd,duedaynbr,dueweekdaycd,coverageamt,firstyearnbr,refundableyn,instypdesc,datelastmaint
2756,5698,2025-01-28,1151,HOME,HP 3212212,NaN,ANNU,1725.0,2020-03-12,2021-02-01,NaN,2021-03-10,2.0,1.0,NaN,587000.0,NaN,N,Homeowner's Insurance,2025-01-28 21:49:25


In [15]:
prop_data[prop_data['propnbr'] == 50885]

,acctnbr,propnbr,propaddr1,propaddr2,propcity,propstate,propzip,aprsvalueamt,aprsdate,proptypdesc,propdesc
55390,150439381,50885,8 HIDDEN HILLS DRIVE,NaN,SEEKONK,MA,2771.0,695000.0,2020-02-21,1 Family Residential - Own Occ,"8 Hidden Hills Drive, Seekonk, MA, 02771"


In [16]:
wh_inspolicy[wh_inspolicy['intrpolicynbr'] == 5697]

,intrpolicynbr,rundate,insorgnbr,instypcd,extpolicynbr,agentorgnbr,premcalperiodcd,premamt,effdate,expiredate,policydocnbr,inactivedate,firstmonthcd,duedaynbr,dueweekdaycd,coverageamt,firstyearnbr,refundableyn,instypdesc,datelastmaint
2755,5697,2025-01-28,1204,PMI,6471333498,NaN,MNTH,82.5,2020-05-01,2027-08-01,NaN,NaN,NaN,1.0,NaN,0.25,NaN,N,Private Mortgage Insurance,2025-01-28 21:49:25


In [17]:
assert acctpropins['intrpolicynbr'].is_unique, "Duplicates"

AssertionError: Duplicates

In [18]:
deduped_acctpropins = acctpropins.drop_duplicates(subset='intrpolicynbr')

In [19]:
wh_acctcommon.head()

,acctnbr,effdate,ownersortname,nameaddr1,nameaddr2,primaryownercity,primaryownerstate,primaryownerzipcd,notebal,bookbalance,mjaccttypcd,currmiaccttypcd,curracctstatcd,taxrptforpersnbr,taxrptfororgnbr,loanofficer,acctofficer,contractdate,noteopenamt,noteintrate
0,103805,2025-01-28,SEM REALTY LLC,SEM REALTY LLC,CVE REALTY LLC,PROVIDENCE,RI,2906.0,1010088.92,0.00,CML,CM73,ACT,NaN,1011792.0,ANDREW J. OMER,NaN,2019-12-03,1158750.0,0.04140
1,103807,2025-01-28,SEM REALTY LLC,SEM REALTY LLC,CVE REALTY LLC,PROVIDENCE,RI,2906.0,1010088.92,1010088.92,CML,CM74,ACT,NaN,1011792.0,ANDREW J. OMER,NaN,2019-12-03,1158750.0,0.06587
2,103813,2025-01-28,LEFAS PARK AVE REALTY INC,LEFAS PARK AVE REALTY INC,21 LONGMEADOW DR,WESTWOOD,MA,2090.0,169950.46,169950.46,CML,CM61,ACT,NaN,1010140.0,SBLC LOAN OFFICER,NaN,2019-12-18,201000.0,0.07920
3,103821,2025-01-28,"RICG-C, LLC",6 BLACKSTONE VALLEY PLACE BUILDING 1,SUITE 105,LINCOLN,RI,2865.0,360573.26,360573.26,CML,CM74,ACT,NaN,1012005.0,ANDREW J. OMER,NaN,2019-12-19,660000.0,0.06587
4,103825,2025-01-28,"CATALAN AUTO AND TRUCK CENTER, INC.","CATALAN AUTO AND TRUCK CENTER, INC.",1045 CRANSTON ST,CRANSTON,RI,2920.0,3939.65,3939.65,CML,CM50,ACT,NaN,1010141.0,SBLC LOAN OFFICER,NaN,2019-12-26,50000.0,0.05000


In [20]:
duplicates = acctpropins[acctpropins.duplicated(subset=['intrpolicynbr'], keep=False)]
duplicates

,acctnbr,propnbr,intrpolicynbr,effdate,inactivedate,datelastmaint,escrowyn,lenderfundedyn,escdisbchangeeffdate
752,151016849,78177,15485,2023-11-28,NaN,2024-03-06 12:32:33,N,N,2024-05-01
753,151016849,78177,13584,2024-03-07,NaN,2024-11-14 11:14:57,Y,N,2024-04-01
2898,970189091,35153,2192,2016-03-02,NaN,2019-11-26 14:59:41,N,N,NaN
3299,150857468,78177,15485,2023-11-28,NaN,2023-11-28 11:02:58,N,N,NaN
6070,150857468,78177,13584,2023-04-01,NaN,2023-03-14 10:24:04,Y,N,2023-04-01
6993,150429077,35153,2192,2020-02-04,NaN,2024-11-20 13:52:06,N,N,NaN


Shown above is a good complex example of the ACCTPROPINS table

In [73]:
prop_data[prop_data['propnbr'] == 78177]

,acctnbr,propnbr,propaddr1,propaddr2,propcity,propstate,propzip,aprsvalueamt,aprsdate,proptypdesc,propdesc
66404,150857468,78177,4234 NORTH MAIN ST UNIT 408,NaN,FALL RIVER,MA,2720.0,270000.0,2023-01-30,Condominium,"4234 North Main St, Unit 408, Fall River, MA, ..."
77265,151016849,78177,4234 NORTH MAIN ST UNIT 408,NaN,FALL RIVER,MA,2720.0,270000.0,2023-01-30,Condominium,"4234 North Main St, Unit 408, Fall River, MA, ..."


In [68]:
wh_inspolicy[wh_inspolicy['intrpolicynbr'] == 15485]

,intrpolicynbr,rundate,insorgnbr,instypcd,extpolicynbr,agentorgnbr,premcalperiodcd,premamt,effdate,expiredate,policydocnbr,inactivedate,firstmonthcd,duedaynbr,dueweekdaycd,coverageamt,firstyearnbr,refundableyn,instypdesc,datelastmaint
97,15485,2025-01-28,1106,HOME,8500069918,NaN,ANNU,1.0,2023-11-28,2024-05-14,NaN,NaN,5.0,1.0,NaN,1000000.0,NaN,N,Homeowner's Insurance,2025-01-28 21:49:25


In [72]:
wh_inspolicy[wh_inspolicy['intrpolicynbr'] == 13584]

,intrpolicynbr,rundate,insorgnbr,instypcd,extpolicynbr,agentorgnbr,premcalperiodcd,premamt,effdate,expiredate,policydocnbr,inactivedate,firstmonthcd,duedaynbr,dueweekdaycd,coverageamt,firstyearnbr,refundableyn,instypdesc,datelastmaint
5760,13584,2025-01-28,1203,PMI,1675759,NaN,MNTH,85.12,2024-03-07,2034-09-01,NaN,NaN,NaN,1.0,NaN,0.18,NaN,N,Private Mortgage Insurance,2025-01-28 21:49:25


In [74]:
wh_acctcommon[wh_acctcommon['acctnbr'] == 151016849]

,acctnbr,effdate,ownersortname,nameaddr1,nameaddr2,primaryownercity,primaryownerstate,primaryownerzipcd,notebal,bookbalance,mjaccttypcd,currmiaccttypcd,curracctstatcd,taxrptforpersnbr,taxrptfororgnbr,loanofficer,acctofficer,contractdate,noteopenamt,noteintrate
20328,151016849,2025-01-28,"GRANTHAM, ROBERT V.",ROBERT V. GRANTHAM,PO BOX 9013,FALL RIVER,MA,2720.0,254591.33,254591.33,MTG,MG30,ACT,1153448.0,NaN,DAVID C. TIPPING,NaN,2024-03-06,261900.0,0.06


In [76]:
wh_acctcommon[wh_acctcommon['acctnbr'] == 150857468]

,acctnbr,effdate,ownersortname,nameaddr1,nameaddr2,primaryownercity,primaryownerstate,primaryownerzipcd,notebal,bookbalance,mjaccttypcd,currmiaccttypcd,curracctstatcd,taxrptforpersnbr,taxrptfororgnbr,loanofficer,acctofficer,contractdate,noteopenamt,noteintrate


This is a closed account (acctpropins table not getting cleared properly I believe)

In [77]:
prop_data[prop_data['propnbr'] == 35153]

,acctnbr,propnbr,propaddr1,propaddr2,propcity,propstate,propzip,aprsvalueamt,aprsdate,proptypdesc,propdesc
35474,970189091,35153,460 MENAUHANT RD,NaN,E FALMOUTH,MA,25366951.0,NaN,NaN,1 Family Residential - Own Occ,"460 MENAUHANT RD, E FALMOUTH, MA 02536"
57275,150429077,35153,460 MENAUHANT RD,NaN,E FALMOUTH,MA,25366951.0,NaN,NaN,1 Family Residential - Own Occ,"460 MENAUHANT RD, E FALMOUTH, MA 02536"


In [78]:
wh_inspolicy[wh_inspolicy['intrpolicynbr'] == 2192]

,intrpolicynbr,rundate,insorgnbr,instypcd,extpolicynbr,agentorgnbr,premcalperiodcd,premamt,effdate,expiredate,policydocnbr,inactivedate,firstmonthcd,duedaynbr,dueweekdaycd,coverageamt,firstyearnbr,refundableyn,instypdesc,datelastmaint
6119,2192,2025-01-28,1105,FLOD,0002385096,NaN,ANNU,1441.0,2020-02-04,2025-11-20,NaN,NaN,11.0,20.0,NaN,250000.0,NaN,N,Flood Insurance,2025-01-28 21:49:25


In [79]:
wh_acctcommon[wh_acctcommon['acctnbr'] == 970189091]

,acctnbr,effdate,ownersortname,nameaddr1,nameaddr2,primaryownercity,primaryownerstate,primaryownerzipcd,notebal,bookbalance,mjaccttypcd,currmiaccttypcd,curracctstatcd,taxrptforpersnbr,taxrptfororgnbr,loanofficer,acctofficer,contractdate,noteopenamt,noteintrate


In [80]:
wh_acctcommon[wh_acctcommon['acctnbr'] == 150429077]

,acctnbr,effdate,ownersortname,nameaddr1,nameaddr2,primaryownercity,primaryownerstate,primaryownerzipcd,notebal,bookbalance,mjaccttypcd,currmiaccttypcd,curracctstatcd,taxrptforpersnbr,taxrptfororgnbr,loanofficer,acctofficer,contractdate,noteopenamt,noteintrate
5833,150429077,2025-01-28,"DUMONT, RAYMOND L.",RAYMOND L. DUMONT,SANDRA H DUMONT,E FALMOUTH,MA,2536.0,0.0,0.0,MTG,MG52,ACT,1032825.0,NaN,BERNADETTE S. SANTOS,NaN,2020-02-04,NaN,0.075


In [64]:
deduped_acctpropins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7957 entries, 0 to 7959
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   acctnbr               7957 non-null   int64 
 1   propnbr               7957 non-null   int64 
 2   intrpolicynbr         7957 non-null   int64 
 3   effdate               7957 non-null   object
 4   inactivedate          2 non-null      object
 5   datelastmaint         7957 non-null   object
 6   escrowyn              7957 non-null   object
 7   lenderfundedyn        7957 non-null   object
 8   escdisbchangeeffdate  6915 non-null   object
dtypes: int64(3), object(6)
memory usage: 621.6+ KB


In [57]:
acctpropins

,acctnbr,propnbr,intrpolicynbr,effdate,inactivedate,datelastmaint,escrowyn,lenderfundedyn,escdisbchangeeffdate
0,201342911,30221,4213,2019-05-01,NaN,2024-05-01 08:07:28,Y,N,2019-05-01
1,150426875,50221,5446,2020-02-05,NaN,2020-07-26 23:28:41,Y,N,2021-02-01
2,201754331,32082,5694,2020-03-11,NaN,2020-07-26 23:28:41,Y,N,2020-03-01
3,150439381,50885,5697,2020-05-01,NaN,2023-02-13 10:53:20,Y,N,2020-05-01
4,150439381,50885,5698,2020-03-12,NaN,2024-12-17 14:41:00,N,N,2021-02-01
...,...,...,...,...,...,...,...,...,...
7955,150741380,71134,10485,2022-05-02,NaN,2024-08-22 08:32:15,Y,N,2022-08-01
7956,150773896,73616,11646,2022-07-29,NaN,2024-06-18 12:13:48,Y,N,2023-07-01
7957,150243584,40256,11764,2022-08-15,NaN,2024-07-22 10:09:46,Y,N,2023-07-01
7958,201253881,29920,11807,2022-08-17,NaN,2024-08-20 08:24:20,Y,N,2023-08-01


In [53]:
wh_inspolicy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7957 entries, 0 to 7956
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   intrpolicynbr    7957 non-null   int64  
 1   rundate          7957 non-null   object 
 2   insorgnbr        7957 non-null   int64  
 3   instypcd         7957 non-null   object 
 4   extpolicynbr     7957 non-null   object 
 5   agentorgnbr      4 non-null      float64
 6   premcalperiodcd  7944 non-null   object 
 7   premamt          7944 non-null   float64
 8   effdate          7957 non-null   object 
 9   expiredate       7466 non-null   object 
 10  policydocnbr     0 non-null      float64
 11  inactivedate     1695 non-null   object 
 12  firstmonthcd     7353 non-null   float64
 13  duedaynbr        7937 non-null   float64
 14  dueweekdaycd     0 non-null      float64
 15  coverageamt      7495 non-null   float64
 16  firstyearnbr     0 non-null      float64
 17  refundableyn  

In [54]:
assert wh_inspolicy['intrpolicynbr'].is_unique, "Duplicates"

In [55]:
assert acctpropins['intrpolicynbr'].is_unique, "Duplicates"

AssertionError: Duplicates

In [33]:
def consolidation_with_multiple_props(main_loan_data, property_data):
    """
    This is the main_loan_data joined with property_data, including multiple properties per acctnbr.
    Keep in mind acctnbr field is not unique here.

    Args:
        main_loan_data (pd.DataFrame)
        property_data (pd.DataFrame)

    Returns:
        df (pd.DataFrame): df with multiple properties per acctnbr
    """
    # Pre-merge validation
    assert main_loan_data['acctnbr'].is_unique, "Duplicates exist"

    # Merge
    df = pd.merge(main_loan_data, property_data, on='acctnbr', how='left')
    return df

In [34]:
merged_data = consolidation_with_multiple_props(main_data, prop_data)

In [35]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24930 entries, 0 to 24929
Data columns (total 49 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   acctnbr                 24930 non-null  int64         
 1   effdate                 24930 non-null  object        
 2   ownersortname           24930 non-null  object        
 3   nameaddr1               24930 non-null  object        
 4   nameaddr2               24930 non-null  object        
 5   primaryownercity        24908 non-null  object        
 6   primaryownerstate       24908 non-null  object        
 7   primaryownerzipcd       24908 non-null  float64       
 8   notebal                 24930 non-null  float64       
 9   bookbalance             24930 non-null  float64       
 10  mjaccttypcd             24930 non-null  object        
 11  currmiaccttypcd         24930 non-null  object        
 12  curracctstatcd          24930 non-null  object

In [36]:
acctpropins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7960 entries, 0 to 7959
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   acctnbr               7960 non-null   int64 
 1   propnbr               7960 non-null   int64 
 2   intrpolicynbr         7960 non-null   int64 
 3   effdate               7960 non-null   object
 4   inactivedate          2 non-null      object
 5   datelastmaint         7960 non-null   object
 6   escrowyn              7960 non-null   object
 7   lenderfundedyn        7960 non-null   object
 8   escdisbchangeeffdate  6916 non-null   object
dtypes: int64(3), object(6)
memory usage: 559.8+ KB


In [37]:
assert acctpropins['acctnbr'].is_unique, "Duplicates found"

AssertionError: Duplicates found

In [38]:
assert acctpropins['propnbr'].is_unique, "Duplicates found"

AssertionError: Duplicates found

In [39]:
acctpropins.head()

,acctnbr,propnbr,intrpolicynbr,effdate,inactivedate,datelastmaint,escrowyn,lenderfundedyn,escdisbchangeeffdate
0,201342911,30221,4213,2019-05-01,NaN,2024-05-01 08:07:28,Y,N,2019-05-01
1,150426875,50221,5446,2020-02-05,NaN,2020-07-26 23:28:41,Y,N,2021-02-01
2,201754331,32082,5694,2020-03-11,NaN,2020-07-26 23:28:41,Y,N,2020-03-01
3,150439381,50885,5697,2020-05-01,NaN,2023-02-13 10:53:20,Y,N,2020-05-01
4,150439381,50885,5698,2020-03-12,NaN,2024-12-17 14:41:00,N,N,2021-02-01


In [40]:
wh_inspolicy = pd.read_csv(Path(r'Z:\Chad Projects\Ad Hoc Reports\Lee_Mason_Ins_Tracking\Production\assets\caching\wh_inspolicy.csv'))

In [41]:
wh_inspolicy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7957 entries, 0 to 7956
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   intrpolicynbr    7957 non-null   int64  
 1   rundate          7957 non-null   object 
 2   insorgnbr        7957 non-null   int64  
 3   instypcd         7957 non-null   object 
 4   extpolicynbr     7957 non-null   object 
 5   agentorgnbr      4 non-null      float64
 6   premcalperiodcd  7944 non-null   object 
 7   premamt          7944 non-null   float64
 8   effdate          7957 non-null   object 
 9   expiredate       7466 non-null   object 
 10  policydocnbr     0 non-null      float64
 11  inactivedate     1695 non-null   object 
 12  firstmonthcd     7353 non-null   float64
 13  duedaynbr        7937 non-null   float64
 14  dueweekdaycd     0 non-null      float64
 15  coverageamt      7495 non-null   float64
 16  firstyearnbr     0 non-null      float64
 17  refundableyn  

In [42]:
wh_inspolicy.head()

,intrpolicynbr,rundate,insorgnbr,instypcd,extpolicynbr,agentorgnbr,premcalperiodcd,premamt,effdate,expiredate,policydocnbr,inactivedate,firstmonthcd,duedaynbr,dueweekdaycd,coverageamt,firstyearnbr,refundableyn,instypdesc,datelastmaint
0,2756,2025-01-28,1114,HOME,002235290-95A,NaN,ANNU,1049.37,2011-09-30,2027-11-01,NaN,NaN,6.0,1.0,NaN,450000.0,NaN,N,Homeowner's Insurance,2025-01-28 21:49:25
1,2757,2025-01-28,1132,HOME,HP3 W956054,NaN,ANNU,541.00,2011-09-30,2041-10-01,NaN,2024-09-30,9.0,1.0,NaN,0.0,NaN,N,Homeowner's Insurance,2025-01-28 21:49:25
2,2758,2025-01-28,1097,HOME,8500049395,NaN,ANNU,1.00,2011-09-30,2041-10-01,NaN,NaN,10.0,1.0,NaN,1000000.0,NaN,N,Homeowner's Insurance,2025-01-28 21:49:25
3,2759,2025-01-28,1097,HOME,8500049395,NaN,ANNU,1.00,2011-09-30,2041-10-01,NaN,NaN,10.0,1.0,NaN,11170000.0,NaN,N,Homeowner's Insurance,2025-01-28 21:49:25
4,2760,2025-01-28,1097,HOME,BCYMLH,NaN,ANNU,0.01,2011-10-11,2031-11-01,NaN,NaN,11.0,1.0,NaN,292000.0,NaN,N,Homeowner's Insurance,2025-01-28 21:49:25


In [43]:
wh_inspolicy['instypcd'].unique()

array(['HOME', 'FLOD', 'PMI', 'FPLC', 'BRSK', 'LIAB', 'COML', 'CNDO',
       'FORP'], dtype=object)

In [44]:
assert wh_inspolicy['intrpolicynbr'].is_unique, "Duplicates"